In [8]:
import pandas as pd

In [11]:
df = pd.read_csv(r'/home/karthik/Desktop/drowsiness_detection_project/drowsiness_data/drowsiness_dataset_cleaned.csv')

In [14]:
df.head()

,window_id,perclos,eye_blink_rate,blink_duration,yawning_rate,steering_entropy,steering_reversal_rate,std_lane_position,drowsiness_level
0,0,1.102088,8.352019,0.082657,0.0,0.089473,0.0,0.185746,1
1,1,1.239437,8.103700,0.092958,0.0,0.044457,3.0,0.203346,1
2,2,2.585722,14.156162,0.110817,0.0,0.053407,3.0,0.192183,1
3,3,2.200903,13.193634,0.094324,0.0,0.053932,0.0,0.233520,1
4,4,1.860203,11.165221,0.101466,0.0,0.019984,1.0,0.213990,1
